# Load data in

In [ ]:
import numpy as np
from pathlib import Path

def reshape_array(arr):
    return arr.reshape(arr.shape[0], arr.shape[1], 1)

def load_data_in(folder_name, prefix):
    current_dir = Path('./')
    data_dir = current_dir / folder_name
    train_data = data_dir / '{}_train_reverb.npy'.format(prefix)
    train_label = current_dir / 'y_train.npy'
    test_data = data_dir / '{}_test_reverb.npy'.format(prefix)
    test_label = current_dir / 'y_test.npy'
    
    with open(train_data, 'rb') as f:
        X_train_data = np.load(f)
        X_train_data = reshape_array(X_train_data)
        
    with open(test_data, 'rb') as f:
        X_test_data = np.load(test_data)
        X_test_data = reshape_array(X_test_data)
        
    with open(train_label, 'rb') as f:
        y_train = np.load(f)
        
    with open(test_label, 'rb') as f:
        y_test = np.load(f)
        
    return X_train_data, y_train, X_test_data, y_test

# E2E Model

In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
        
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

def generate_e2e_model():
    fs = 16000
    model_e2e = tf.keras.models.Sequential()
    model_e2e.add(tf.keras.Input(shape=(fs,1))) #Make sure that the input size is the size of the signal
    model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean
    #First Conv1D layer
    model_e2e.add(tf.keras.layers.Conv1D(8,13, padding='valid', activation='relu', strides=1)) #Convolve with 8 1D kernels of length 13
    model_e2e.add(tf.keras.layers.MaxPooling1D(3)) #Downsample - take the max out of every three elements
    model_e2e.add(tf.keras.layers.Dropout(0.3)) #drop nodes with probability 0.3
    #Second Conv1D layer
    model_e2e.add(tf.keras.layers.Conv1D(16, 11, padding='valid', activation='relu', strides=1)) #Convolve with 16 1D kernels of length 11
    model_e2e.add(tf.keras.layers.MaxPooling1D(3))
    model_e2e.add(tf.keras.layers.Dropout(0.3))
    #Third Conv1D layer
    model_e2e.add(tf.keras.layers.Conv1D(32, 9, padding='valid', activation='relu', strides=1))
    model_e2e.add(tf.keras.layers.MaxPooling1D(3))
    model_e2e.add(tf.keras.layers.Dropout(0.3))
    model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True))
    model_e2e.add(tf.keras.layers.Bidirectional(tf.python.keras.layers.CuDNNGRU(128, return_sequences=True), merge_mode='sum')) #Recurrent layer, uses time series data
    model_e2e.add(tf.keras.layers.Bidirectional(tf.python.keras.layers.CuDNNGRU(128, return_sequences=True), merge_mode='sum'))
    model_e2e.add(tf.keras.layers.Bidirectional(tf.python.keras.layers.CuDNNGRU(128, return_sequences=False), merge_mode='sum')) #set return sequences to False for last recurrent layer
    model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True))
    #Flatten layer
    model_e2e.add(tf.keras.layers.Flatten()) # Turn 2D result of convolution to a single vector
    #Dense Layer 1
    model_e2e.add(tf.keras.layers.Dense(256, activation='relu')) #Fully connected layer
    model_e2e.add(tf.keras.layers. Dense(10, activation="softmax")) #output layer, need size = num_classes
    model_e2e.summary() #show breakdown of parameters

    model_e2e.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy']) #decide loss function and metrics
    
    return model_e2e

def evaluate_approach(folder_name, prefix):
    X_train, y_train, X_test, y_test = load_data_in(folder_name, prefix)
    print(X_test.shape, y_test.shape)
    print(X_train.shape, y_train.shape)

    model = generate_e2e_model()
    
    hist = model.fit(x=X_train, y=y_train, epochs=50, batch_size=32)
    model.evaluate(X_test, y_test)
    
    y_pred = model.predict(X_test, batch_size=32)
    y_pred = (y_pred > 0.5) 
    
    return model, classification_report(y_test, y_pred)

In [ ]:
model, report = evaluate_approach('E2E_data', 'X')
print(report)

In [ ]:
model, report = evaluate_approach('E2E_data', 'Vnet')
print(report)

In [ ]:
model, report = evaluate_approach('E2E_data', 'AE')
print(report)

In [ ]:
model, report = evaluate_approach('E2E_data', 'WPE')
print(report)

In [ ]:
model, report = evaluate_approach('E2E_data', 'ConvTasnet')
print(report)